# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
!pip install langchain_community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify

# the choice of the value for the temperature parameter will be 1 becouse it is a good value for creativity and relability 
llm = ChatOpenAI(temperature=1)


In [9]:
prompt = ChatPromptTemplate.from_template(
    "Describe the following product: {product}"
)

In [10]:

chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/ts/c0z20d1s1rq1pd4h0py597pw0000gn/T/ipykernel_18486/546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
# chosese a product from the dataset
product = df.iloc[0]['Product']
chain.run(product)

/var/folders/ts/c0z20d1s1rq1pd4h0py597pw0000gn/T/ipykernel_18486/2698468722.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'A Queen Size Sheet Set typically includes one flat sheet, one fitted sheet, and two standard pillowcases, all designed to fit a queen size bed. The sheets are typically made of soft and comfortable materials such as cotton, microfiber, or linen. They come in various colors and patterns to suit different bedroom decors. The fitted sheet has elasticized corners to ensure a secure fit on the mattress, while the flat sheet adds an extra layer of comfort and style. The pillowcases feature envelope closures to keep pillows in place. A Queen Size Sheet Set is an essential bedding item for any queen size bed, providing a cozy and stylish sleep environment.'

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
     "Describe the following product: {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the description of the product: {product_description}, write a short advertisement for it."
    
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A queen size sheet set typically includes a fitted sheet, flat sheet, and pillowcases designed to fit a queen size bed. The sheets are usually made of soft and durable materials such as cotton, microfiber, or bamboo, providing a comfortable and snug fit for the mattress. The set may come in various colors, patterns, and thread counts to suit different preferences and styles. Queen size sheet sets are commonly used in bedrooms to provide a cozy and luxurious sleeping experience.
Upgrade your bedroom with our luxurious queen size sheet set! Made of soft and durable materials like cotton, microfiber, or bamboo, our sheet set includes a fitted sheet, flat sheet, and pillowcases designed to fit a queen size bed perfectly. Choose from a variety of colors, patterns, and thread counts to suit your style and preferences. Transform your bedtime routine into a comfortable and cozy experience with our queen size sheet set. Order yours today and sleep

'Upgrade your bedroom with our luxurious queen size sheet set! Made of soft and durable materials like cotton, microfiber, or bamboo, our sheet set includes a fitted sheet, flat sheet, and pillowcases designed to fit a queen size bed perfectly. Choose from a variety of colors, patterns, and thread counts to suit your style and preferences. Transform your bedtime routine into a comfortable and cozy experience with our queen size sheet set. Order yours today and sleep like royalty every night!'

**Repeat the above twice for different products**

## SequentialChain

In [18]:
from langchain.chains import SequentialChain

In [23]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review into English: {Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated_review" #Give a name to your output
                    )


In [26]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a prompt to summarize a review"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary_prompt" #give a name to this output
                    )


In [27]:
# prompt template 3: translate to a specified language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into {language}: {Review}"
)
# chain 3: input= Review and language, output= translated_review
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_review"
                      )


In [28]:

# prompt template 4: follow up message that takes as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Based on the translated review: {translated_review} and the summary prompt: {summary_prompt}, write a follow-up message."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [32]:
# overall_chain: input= Review 
# and output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_three, chain_two, chain_four],  # Adjust the order of chains
    input_variables=["Review", "language"],  # Specify the input variables
    output_variables=["translated_review", "summary_prompt", "followup_message"],  # Specify the output variables
    verbose=True
)

In [34]:
review = df.Review[5]
language = "English"  # Specify the target language
overall_chain({"Review": review, "language": language})



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'language': 'English',
 'translated_review': "I find the taste mediocre. The foam does not hold, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary_prompt': 'Please summarize your experience with the product or service in a few sentences. Include your overall thoughts, key points, and whether or not you would recommend it to others.',
 'followup_message': "Summary: Overall, the reviewer found the taste of the product to be mediocre and noted that the foam did not hold up well. They questioned whether the product they purchased was an old batch or counterfeit, as they have had a better experience with the same product purchased in stores. The reviewer is left feeling disappointed and unsure about the authenticity of the product.\n\nFollow-up message

**Repeat the above twice for different products or reviews**

## Router Chain

In [35]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [36]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [37]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [38]:
llm = ChatOpenAI(temperature=0)

In [39]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [40]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [41]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [42]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [43]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/ts/c0z20d1s1rq1pd4h0py597pw0000gn/T/ipykernel_18486/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [44]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [45]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [46]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**